In [1]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator

import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio

#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
    horizon = 5
)

query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
query_hello="Hello"
import pickle
with open('queries.pkl', 'rb') as file:
    queries = pickle.load(file)



parallel_calling_agent.clear_logs()
for _ in range(5):
    await parallel_calling_agent.run_agent_trials_async(num_trials=3, 
    query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
    reference="The former former president of Korea is older. The sum of their age is 99.")




[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea age), Tool 2-> YDCSearch(query=MLB American League MVP last season age)]
[INFO] (step 1-observation) :: [Tool 1-> As of the last update I have, the most recent former president of South Korea is Moon Jae-in, who was born on January 24, 1953. Therefore, as of April 2023, Moon Jae-in would be 70 years old. If you are asking about a different former president, please specify which one, and I can provide their age if the information is available up to my last update., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani. His age during the 2023 season was 28 years old, as he was born on July 5, 1994.]
[INFO] (step 2-action) Tool invocations in parallel: Now I can answer directly without resort

RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 1-observation) :: [Tool 1-> Park Geun-hye is a former president of South Korea who served from 2013 until her impeachment in 2017. She was the country's first female president and the first president to be born after the founding of South Korea. Her presidency ended when she was impeached and subsequently convicted on charges related to corruption. After spending nearly five years in prison, Park Geun-hye was released in December 2021 following a presidential pardon., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani of the LA Angels.]
[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=Park Geun-hye age), Tool 2-> YDCSearch(query=Shohei Ohtani age)]
[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952. As of my knowledge cutoff in 2023, she would be 71 years old., Tool 2-> Shohei Ohtani is 29 years old.]
[INFO] (step 3-action) Tool invocations in parallel: Now I can answer directly without reso

RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 1-observation) :: [Tool 1-> As of my knowledge cutoff date in April 2023, the most recent former president of South Korea is Moon Jae-in, who was born on January 24, 1953. Therefore, as of April 2023, Moon Jae-in would be 70 years old. Please note that ages will change as time passes, so you may need to calculate the current age based on the birth date provided., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani. His age as of the 2023 season was 28 years old, as he was born on July 5, 1994.]
[INFO] (step 2-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: Moon Jae-in, the former president of Korea, is older than Shohei Ohtani, the MLB American League MVP of the last season. The sum of their ages is 70 (Moon Jae-in) + 28 (Shohei Ohtani) = 98 years old.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "The former former president of Korea is older. The sum of their age is 99."


...

[INFO] (step 1-observation) :: [Tool 1-> The age of a former president of South Korea would depend on which former president you are referring to and the current date. Here are the ages of the living former presidents as of my knowledge cutoff in 2023:

1. Lee Myung-bak (이명박) - Born December 19, 1941, he would be 81 years old.
2. Park Geun-hye (박근혜) - Born February 2, 1952, she would be 71 years old.

Please note that ages would need to be adjusted based on the current date., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani. Since his birthdate is July 5, 1994, he was 29 years old when he won the MVP award last season.]
[INFO] (step 2-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: The former president of South Korea, Lee Myung-bak, is older than the MLB American League MVP of the last season, Shohei Ohtani. Lee Myung-bak is 81 years old while Shohei Ohtani is 29 years old.

(81 + 29) = 110

Sum of thei

RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952. As of my knowledge cutoff in 2023, she would be 71 years old., Tool 2-> Shohei Ohtani is 29 years old.]
[INFO] (step 3-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: Park Geun-hye is older, and the sum of their ages is 100.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "The former former president of Korea is older. The sum of their age is 99."
[INFO] Trial 3
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea age), Tool 2-> YDCSearch(query=MLB American League MVP last season age)]


RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 1-observation) :: [Tool 1-> As of my knowledge cutoff date in April 2023, the most recent former president of South Korea is Moon Jae-in, who was born on January 24, 1953. Therefore, as of April 2023, Moon Jae-in would be 70 years old. Please note that ages will change as time passes, so you may need to calculate the current age based on the birth date provided., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani. Since his birthdate is July 5, 1994, he was 29 years old when he won the award.]
[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> PythonREPL(command=print(70 + 29))]
[INFO] (step 2-observation) :: [Tool 1-> 99]
[INFO] (step 3-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: The former president of Korea, Moon Jae-in, is older than the MLB American League MVP of the last season, Shohei Ohtani. When you sum up the ages of the two, the result is 99.
[INFO] Jugdement: Your 